In [ ]:
%matplotlib notebook
import time
from tqdm import tqdm
import sys
from bs4 import BeautifulSoup
import requests
import csv
import re
import math
import os.path
import xmltodict
import pandas as pd
import shutil
import math
from tqdm import tqdm_notebook
from os import listdir, path
from PIL import Image
from shutil import copyfile
import numpy as np
from matplotlib import pyplot as plt
from langdetect import detect

In [ ]:
# put your api key here
GOODREADS_API_KEY = 'sMrgsSTFjbGpoqLuG29Yeg'
GOODREADS_API_URL = 'https://www.goodreads.com/'
en_wiki_page_url = 'https://en.wikipedia.org/wiki/List_of_Nobel_laureates_in_Literature'
fa_wiki_page_url = 'https://fa.wikipedia.org/wiki/%D9%81%D9%87%D8%B1%D8%B3%D8%AA_%D8%A8%D8%B1%D9%86%D8%AF%DA%AF%D8%A7%D9%86_%D8%AC%D8%A7%DB%8C%D8%B2%D9%87_%D9%86%D9%88%D8%A8%D9%84_%D8%A7%D8%AF%D8%A8%DB%8C%D8%A7%D8%AA'
iranketab_url = 'https://www.iranketab.ir'
authors_file_path = 'data/authors.csv'
orginal_books_dir = 'data/orginal_books/'
orginal_books_file_path = 'data/orginal-books.csv'
persian_books_dir = 'data/persian-books/'
books_file_path = 'books.csv'
authors_image_path = 'output/images/authors/'
books_image_path = 'output/images/books/'

In [ ]:
def make_api_request(url):
    url = f"{GOODREADS_API_URL}{url}&key={GOODREADS_API_KEY}"
    req = requests.get(url)
    return xmltodict.parse(req.content)['GoodreadsResponse']

def get_image(url,height,save_dir,file_name):
     image_type = url[len(url)-4:] 
     save_path = f'{save_dir}{file_name}{image_type}'
     if path.isfile(save_path):
         return f"{file_name}{image_type}"
        
     image_url = url
     image_url_parts = url.split('._')
     
     if len(image_url_parts) == 2:
        image_url = image_url_parts[0] + image_url_parts[1][len(image_url_parts[1])-4:]    
     
     temp_file = f"temp{image_type}"
     download_image(image_url,temp_file)

     try:
         img = Image.open(temp_file)
         new_width  = int(height * img.size[0] / img.size[1])
         img = img.resize((new_width, height), Image.ANTIALIAS)
         img.save(f'{save_dir}{file_name}{image_type}')   
     except Exception as e:
        if path.isfile(temp_file):
            copyfile(temp_file, f'{save_dir}{file_name}.{image_type}')
     return f"{file_name}{image_type}"

def write_to_csv(file_path, data):
    with open(file_path, 'w', newline='') as file:
        writer = csv.writer(file, delimiter=',')
        for j in data:
            writer.writerow(j)

<h1>Authors</h1>

In [ ]:
# search author by name and return author id            
def get_author_goodreads_id(name):
    # name = quote(name)
    response = make_api_request(f"api/author_url/{name}?")
    return response['author']['@id']


# get author by id
def get_author_info_from_goodreads(author_id):
    response = make_api_request(
        f"author/show/{author_id}.xml?v=2&shelf=read&per_page=5")
    author_data = response['author']
    books_ids = []
    if not isinstance(author_data['books']['book'], list):
        books_ids.append(author_data['books']['book']['id']['#text'])
    else:
        for book in author_data['books']['book']:
            books_ids.append(book['id']['#text'])

    author_info = {
        'id': author_data['id'],
        'name': author_data['name'],
        'url': author_data['link'],
        'large_image_url': author_data['large_image_url'],
        'small_image_url': author_data['small_image_url'],
        'image_url': author_data['image_url'],
        'fans_count': author_data['fans_count'].get('#text', '0'),
        'author_followers_count': author_data['author_followers_count'].get('#text', '0'),
        'works_count': author_data['works_count'],
        'gender': author_data['gender'],
        'hometown': author_data['hometown'],
        'born_at': author_data['born_at'],
        'died_at': author_data['died_at'],
        'about': author_data['about'],
        'books_ids': books_ids
    }
    return author_info

#extract author info from wikipedia table columns
def get_author_info_from_wikipedia(columns):
    image = columns[1].find('img')
    image_url = ""
    if image != None:
        image_url = image.attrs['src']

    author_name_tag = columns[2].find('a')
    name = author_name_tag.text
    about_page_url = author_name_tag.attrs['href']

    countries_tag = columns[3].find_all('a')
    countries = []
    for country in countries_tag:
        countries.append(country.text)

    flags_tgs = columns[3].find_all('img')
    countries_flags = []
    for flag in flags_tgs:
        countries_flags.append(f"https:{flag.attrs['src']}")

    languages_tag = columns[4].find_all('a')
    languages = []
    for language in languages_tag:
        languages.append(language.text)

    citation = columns[5].text
    genres = columns[6].text

    return {
        'name': name,
        'image_url': f'https:{image_url}',
        'about_page_url': about_page_url,
        'countries': countries,
        'countries_flags': countries_flags,
        'languages': languages,
        'citation': citation,
        'genres': genres
    }


# get author data from iranketab site
def get_author_info_from_iranketab(year):
    req = requests.get(
        "https://www.iranketab.ir/tag/25-nobel-prize-in-literature")
    soup = BeautifulSoup(req.content, "html.parser")
    years_tag = soup.find_all('span', {
                              'style': 'position: absolute;top: 80px;left: 5px;padding: 7px 50px;background-color: #ffffff;z-index: 2;border: 1px solid #ccc;font-size: 20px;'})
    for year_tag in years_tag:

        if int(year_tag.text.strip()) == int(year):
            author_name_tag = year_tag.parent.find(
                'p', {'class': 'product-name-title'}).find('a')
            href = iranketab_url + author_name_tag.attrs['href']
            profile_req = requests.get(href)
            profile_soup = BeautifulSoup(profile_req.content, "html.parser")
            about = profile_soup.find(
                'h5', {'style': 'font-size: 15px;line-height: 30px;'}).text.strip()
            image_url = iranketab_url + \
                profile_soup.find('img', {'class': 'img-circle'}).attrs['src']

            return {
                'about': about,
                'image_url': image_url,
                'page_url': href
            }
    return None

def get_authors():
    en_page_response = requests.get(en_wiki_page_url)
    soup_en_page = BeautifulSoup(en_page_response.content, "html.parser")
    en_authors_table = soup_en_page.find(
        "table", {"class": "wikitable sortable"})

    fa_page_response = requests.get(fa_wiki_page_url)
    soup_fa_page = BeautifulSoup(fa_page_response.content, "html.parser")
    fa_authors_table = soup_fa_page.find(
        "table", {"class": "wikitable sortable"})
    authors = [
        [
            'year',
            'gr_id',
            'wp_en_name',
            'wp_en_author_page_url',
            'wp_en_countries',
            'wp_en_languages',
            'wp_en_citation',
            'wp_en_genres',
            'wp_fa_name',
            'wp_fa_author_page_url',
            'wp_fa_countries',
            'wp_fa_languages',
            'wp_fa_citation',
            'wp_fa_genres',
            'wp_image_url',
            'wp_countries_flags',
            'gr_image_url',
            'gr_author_page_url',
            'gr_fans_count',
            'gr_author_followers_count',
            'gr_works_count',
            'gr_gender',
            'gr_hometown',
            'gr_born_at',
            'gr_died_at',
            'gr_about',
            'gr_books_ids',
            'ik_about',
            'ik_image_url',
            'ik_page_url',
            'image_name'
        ]
    ]
    
    pbar = tqdm_notebook(total=len(en_authors_table.find_all("tr")))
    row = 0
    for en_row, fa_row in zip(en_authors_table.find_all("tr"), fa_authors_table.find_all("tr")):
        en_columns = en_row.findAll('td')
        if(len(en_columns) == 0):
            continue

        if(len(en_columns) != 7):
            continue

        fa_columns = fa_row.findAll('td')

        year = en_columns[0].text.strip()[0:4]

        en_author = get_author_info_from_wikipedia(en_columns)
        fa_author = get_author_info_from_wikipedia(fa_columns)

        author_id = get_author_goodreads_id(en_author['name'])
        goodreads_author_info = get_author_info_from_goodreads(author_id)

        iranketab_author_info = get_author_info_from_iranketab(year)
        authors.append([
            year,
            author_id,
            en_author['name'],
            en_author['about_page_url'],
            ",".join(en_author['countries']),
            ",".join(en_author['languages']),
            en_author['citation'],
            en_author['genres'],
            fa_author['name'],
            fa_author['about_page_url'],
            ",".join(fa_author['countries']),
            ",".join(fa_author['languages']),
            fa_author['citation'],
            fa_author['genres'],
            en_author['image_url'],
            ",".join(en_author['countries_flags']),

            goodreads_author_info['image_url'],
            goodreads_author_info['url'],
            goodreads_author_info['fans_count'],
            goodreads_author_info['author_followers_count'],
            goodreads_author_info['works_count'],
            goodreads_author_info['gender'],
            goodreads_author_info['hometown'],
            goodreads_author_info['born_at'],
            goodreads_author_info['died_at'],
            goodreads_author_info['about'],
            ",".join(goodreads_author_info['books_ids']),

            "" if iranketab_author_info == None else iranketab_author_info['about'],
            "" if iranketab_author_info == None else iranketab_author_info['image_url'],
            "" if iranketab_author_info == None else iranketab_author_info['page_url'],
            ""

        ])
        row = row + 1
        pbar.update(1)
    return authors

def save_auhtor_images():
    df = pd.read_csv(authors_file_path)
    df.fillna('',inplace=True)
    for index in range(len(df)):
        image_url = ''
        if df.loc[index,'gr_image_url'] != '' and df.loc[index,'gr_image_url'].find('nophoto')<0:
            image_url = df.loc[index,'gr_image_url']
        elif df.loc[index,'ik_image_url'] != '':
            image_url = df.loc[index,'ik_image_url']
        else:
            image_url = df.loc[index,'wp_image_url'] 
        df.loc[index,'image_name'] =  get_image(image_url,350, authors_image_path,df.loc[index,'gr_id'])   
    df.to_csv(authors_file_path,index=False)                                


In [ ]:
authors = get_authors()
write_to_csv(authors_file_path,authors)
print("Downloaded authors info")
save_auhtor_images()

<h1>Authors Books</h1>

In [ ]:
# get book info from goodreads
def get_book_info(author_id, book_id):
    book_tree = make_api_request(f"book/show/{book_id}.xml?")
    info = book_tree['book']
    translators = {'id': [], 'name': [], 'average_rating': []}
    authors = {'id': [], 'name': [], 'average_rating': []}
    if isinstance(info['authors']['author'], list):
        for author in info['authors']['author']:
            if author['role'] != None and author['role'] == 'Translator':
                translators['id'].append(author['id'])
                translators['name'].append(author['name'])
                translators['average_rating'].append(author['average_rating'])
            else:
                authors['id'].append(author['id'])
                authors['name'].append(author['name'])
                authors['average_rating'].append(author['average_rating'])
    else:
        author = info['authors']['author']
        if author['role'] != None and author['role'] == 'Translator':
            translators['id'].append(author['id'])
            translators['name'].append(author['name'])
            translators['average_rating'].append(author['average_rating'])
        else:
            authors['id'].append(author['id'])
            authors['name'].append(author['name'])
            authors['average_rating'].append(author['average_rating'])

    return {
        'id':info['id'],
        'author_id': author_id,
        'title': info['title'],
        'isbn': info['isbn'],
        'isbn13': info['isbn13'],
        'country_code': info['country_code'],
        'image_url': info['image_url'],
        'small_image_url': info['small_image_url'],
        'publication_year': info['publication_year'],
        'publication_month': info['publication_month'],
        'publication_day': info['publication_day'],
        'language_code': info['language_code'],
        'publisher': info['publisher'],
        'is_ebook': info['is_ebook'],
        'work_id': info['work']['id']['#text'],
        'total_editions_count': info['work']['books_count']['#text'],
        'total_added_by': info['work']['reviews_count']['#text'],
        'total_rating_sum': info['work']['ratings_sum']['#text'],
        'total_rating_count': info['work']['ratings_count']['#text'],
        'total_text_reviews_count': info['work']['text_reviews_count']['#text'],
        'work_rating_dist': info['work']['rating_dist'],
        'work_original_publication_year': info['work']['original_publication_year'].get('#text', ''),
        'work_original_publication_month': info['work']['original_publication_month'].get('#text', ''),
        'work_original_publication_day': info['work']['original_publication_day'].get('#text', ''),
        'avg_rating': info['average_rating'],
        'ratings_count': info['ratings_count'],
        'text_reviews_count': info['text_reviews_count'],
        'authors_ids': authors['id'],
        'authors_names': authors['name'],
        'authors_avg_rating': authors['average_rating'],
        'translators_ids': translators['id'],
        'translators_names': translators['name'],
        'translators_avg_rating': translators['average_rating'],
        'description': info['description']
    }


# get an author's books
def get_author_book_list(author_id):
    #name = quote(name)
    review_tree = make_api_request(f"author/list/{author_id}?per_page=10000")
    return review_tree['author']

def get_books():
    df = pd.read_csv("authors.csv")
    df['ik_page_url'].fillna('', inplace=True)

    year = 0
    try:
        for i in tqdm_notebook(range(len(df))):
            books = [['id', 'author_id', 'title', 'isbn', 'isbn13', 'country_code', 'image_url', 'small_image_url', 'publication_year', 'publication_month', 'publication_day', 'language_code', 'publisher', 'is_ebook', 'work_id', 'total_editions_count', 'total_added_by', 'total_rating_sum', 'total_rating_count', 'total_text_reviews_count',
                      'rating_dist', 'original_publication_year', 'original_publication_month', 'original_publication_day', 'average_rating', 'ratings_count', 'text_reviews_count', 'authors_ids', 'authors_names', 'authors_average_rating', 'translators_ids', 'translators_names', 'translators_average_rating', 'description']]

            year = df.loc[i, 'year']
            if os.path.isfile(f"{orginal_books_dir}{year}.csv"):
                continue

            try:
                for book in get_author_book_list(df.loc[i, 'gr_id'])['books']['book']:

                    book_id = book['id']['#text']

                    try:
                        find = False
                        if isinstance(book['authors']['author'], list):
                            for author in book['authors']['author']:
                                if df.loc[i, 'gr_id'] == author['id']:
                                    find = True
                        else:
                            if int(df.loc[i, 'gr_id']) == int(book['authors']['author']['id']):
                                find = True
                        if find == False:
                            continue
                        
                        book_info = get_book_info(df.loc[i, 'gr_id'], book_id)
                        books.append([
                            book_info['id'],
                            book_info['author_id'],
                            book_info['title'],
                            book_info['isbn'],
                            book_info['isbn13'],
                            book_info['country_code'],
                            book_info['image_url'],
                            book_info['small_image_url'],
                            book_info['publication_year'],
                            book_info['publication_month'],
                            book_info['publication_day'],
                            book_info['language_code'],
                            book_info['publisher'],
                            book_info['is_ebook'],
                            book_info['work_id'],
                            book_info['total_editions_count'],
                            book_info['total_added_by'],
                            book_info['total_rating_sum'],
                            book_info['total_rating_count'],
                            book_info['total_text_reviews_count'],
                            book_info['work_rating_dist'],
                            book_info['work_original_publication_year'],
                            book_info['work_original_publication_month'],
                            book_info['work_original_publication_day'],
                            book_info['avg_rating'],
                            book_info['ratings_count'],
                            book_info['text_reviews_count'],
                            book_info['authors_ids'],
                            book_info['authors_names'],
                            book_info['authors_avg_rating'],
                            book_info['translators_ids'],
                            book_info['translators_names'],
                            book_info['translators_avg_rating'],
                            book_info['description']
                        ])
                        time.sleep(2)
                    except Exception as e:
                        pass
                write_to_csv(f"{orginal_books_dir}{year}.csv", books)
            except Exception as e:
                pass

    except Exception as e:
        pass
    return books


In [ ]:
get_books()
print("downloaded orignal books")

<h1>Persian Editions</h1>

In [ ]:
def get_persian_editions(author_id, work_id, work_count):
    try:
        req = requests.get(
            f"{GOODREADS_API_URL}work/editions/{work_id}?per_page={work_count}")
        soup = BeautifulSoup(req.content, 'html.parser')

        work_editions = soup.find(
            'div', {'class': 'workEditions'})

        elementList = work_editions.find_all(
            'div', {'class': 'elementList clearFix'})
        persian_editons = []
        for book in elementList:
            persian_edtions = book.findAll(lambda tag:  'Persian' in tag.text)

            for edition in persian_edtions:

                if not edition.has_attr('class'):
                    continue

                if edition.attrs['class'][0] != 'editionData':
                    continue

                rating_div = book.find(lambda tag:  tag.text.strip(
                ) == 'Average rating:').parent.findChildren()[1].get_text().strip()
                rating_numbers = re.findall('\d+\.*\d*', rating_div)
                avg_rating = rating_numbers[0]
                rating_count = rating_numbers[1]

                book_link = edition.find(
                    'a', {'class': 'bookTitle'}).attrs['href']
                book_id = re.search(r"\d+", book_link).group(0)

                persian_editons.append(get_book_info(author_id, book_id))
                time.sleep(4)
        return persian_editons
    except Exception as e:
        pass


def get_persian_books():
     for filename in listdir(orginal_books_dir):
       
        filepath = os.path.join(orginal_books_dir, filename)
        if os.path.isfile(f"{persian_books_dir}persain_books_{filename}.csv"):
            continue
        df = pd.read_csv(filepath)
        persian_editons = []
        for i in tqdm_notebook(range(len(df))):
            try:
                books = get_persian_editions(
                    df.loc[i, "author_id"], df.loc[i, "work_id"], df.loc[i, "total_editions_count"])
                if books != None:
                    persian_editons = persian_editons + books
            except Exception as e:
                pass

            # persian_editons = persian_editons + get_persian_editions()

        books = [['id', 'author_id', 'title', 'isbn', 'isbn13', 'country_code', 'image_url', 'small_image_url', 'publication_year', 'publication_month', 'publication_day', 'language_code', 'publisher', 'is_ebook', 'work_id', 'total_editions_count', 'total_added_by', 'total_rating_sum', 'total_rating_count', 'total_text_reviews_count',
                  'rating_dist', 'original_publication_year', 'original_publication_month', 'original_publication_day', 'average_rating', 'ratings_count', 'text_reviews_count', 'authors_ids', 'authors_names', 'authors_average_rating', 'translators_ids', 'translators_names', 'translators_average_rating', 'description']]
        for book_info in persian_editons:
            books.append([
                book_info['id'],
                book_info['author_id'],
                book_info['title'],
                book_info['isbn'],
                book_info['isbn13'],
                book_info['country_code'],
                book_info['image_url'],
                book_info['small_image_url'],
                book_info['publication_year'],
                book_info['publication_month'],
                book_info['publication_day'],
                book_info['language_code'],
                book_info['publisher'],
                book_info['is_ebook'],
                book_info['work_id'],
                book_info['total_editions_count'],
                book_info['total_added_by'],
                book_info['total_rating_sum'],
                book_info['total_rating_count'],
                book_info['total_text_reviews_count'],
                book_info['work_rating_dist'],
                book_info['work_original_publication_year'],
                book_info['work_original_publication_month'],
                book_info['work_original_publication_day'],
                book_info['avg_rating'],
                book_info['ratings_count'],
                book_info['text_reviews_count'],
                ",",join(book_info['authors_ids']),
                ",",join(book_info['authors_names']),
                ",",join(book_info['authors_avg_rating']),
                ",",join(book_info['translators_ids']),
                ",",join(book_info['translators_names']),
                ",",join(book_info['translators_avg_rating']),
                book_info['description']
            ])
        write_to_csv(f"{persian_books_dir}persain_books_{filename}.csv", books)
     print("downloaded persian books")

In [ ]:
get_persian_books()

<h1>Generate authors page</h1>

In [ ]:
def conver_to_persian_digits(string):
    digits_map = {
        "0": "۰",
        "1": "۱",
        "2": "۲",
        "3": "۳",
        "4": "۴",
        "5": "۵",
        "6": "۶",
        "7": "۷",
        "8": "۸",
        "9": "۹",
    }
    return "".join([digits_map[digit] if digit in digits_map.keys() else digit for digit in str(string)])

def download_image(url, path):
    req = requests.get(url, stream=True)
    with open(path, 'wb') as out_file:
        shutil.copyfileobj(req.raw, out_file)
        
def get_image(url,height,file_name):
     image_type = url[len(url)-4:] 
     save_path = f'{books_image_path}{file_name}{image_type}'
     if path.isfile(save_path):
         return f"{file_name}{image_type}"
        
     image_url = url
     image_url_parts = url.split('._')
     
     if len(image_url_parts) == 2:
        image_url = image_url_parts[0] + image_url_parts[1][len(image_url_parts[1])-4:]    
     
     temp_file = f"temp{image_type}"
     download_image(image_url,temp_file)

     try:
         img = Image.open(temp_file)
         new_width  = int(height * img.size[0] / img.size[1])
         img = img.resize((new_width, height), Image.ANTIALIAS)
         img.save(f'{books_image_path}{file_name}{image_type}')   
     except Exception as e:
        if path.isfile(temp_file):
            copyfile(temp_file, f'{books_image_path}{file_name}.{image_type}')
     return f"{file_name}{image_type}"
    

def concatenate_orginal_persian_books():
    books = [[
        'author_id',
        'id',
        'work_id',
        'org_id',
        'org_title',
        'org_publisher',
        'original_publication_year',
        'title',
        'isbn',
        'isbn13',
        'publisher',
        'publication_year',
        'total_editions_count',
        'total_added_by',
        'total_rating_sum',
        'total_rating_count',
        'total_text_reviews_count',
        'score_5',
        'score_4',
        'score_3',
        'score_2',
        'score_1',
        'score_5_pct',
        'score_4_pct',
        'score_3_pct',
        'score_2_pct',
        'score_1_pct',        
        'average_rating',
        'ratings_count',
        'text_reviews_count',
        'authors_average_rating',
        'translators_ids',
        'translators_names',
        'translators_average_rating',
        'description',
        'org_image_name',
        'image_name'
    ]]
    for book_file_name in listdir(orginal_books_dir):
        en_df = pd.read_csv(path.join(orginal_books_dir, book_file_name))
        pr_df = pd.read_csv(
            path.join(persian_books_dir, f"persain_books_{book_file_name}.csv"))
        en_df.fillna('',inplace = True)
        pr_df.fillna('',inplace = True)
        
        if len(pr_df) == 0:
            continue

        en_df.sort_values(by=['total_added_by','ratings_count'],ascending = (False, False),inplace = True)
        for i in range(len(en_df)):
            work_id = en_df.loc[i, 'work_id']
            work_df = pr_df[pr_df.work_id == work_id]

            for index, row in work_df.iterrows():

                if not str(row.author_id) in row.authors_ids:
                    continue

                rating_dist = row['rating_dist'].split('|')
                score_5 = int(rating_dist[0].split(':')[1])
                score_4 = int(rating_dist[1].split(':')[1])
                score_3 = int(rating_dist[2].split(':')[1])
                score_2 = int(rating_dist[3].split(':')[1])
                score_1 = int(rating_dist[4].split(':')[1])
                
                score_5_pct = score_4_pct = score_3_pct = score_2_pct = score_1_pct = 0
                total_rating_count = score_5 + score_4 + score_3 + score_2 + score_1
                if total_rating_count != 0:
                    score_5_pct = round(score_5 * 100 / total_rating_count)
                    score_4_pct = round(score_4 * 100 / total_rating_count)
                    score_3_pct = round(score_3 * 100 / total_rating_count)
                    score_2_pct = round(score_2 * 100 / total_rating_count)
                    score_1_pct = round(score_1 * 100 / total_rating_count)
                
                
                books.append([
                    row['author_id'],
                    row['id'],
                    row['work_id'],
                    en_df.loc[i, 'id'],
                    en_df.loc[i, 'title'],
                    en_df.loc[i, 'publisher'],
                    '' if row['original_publication_year'] == '' else str(row['original_publication_year'])[:4],
                    row['title'],
                    row['isbn'],
                    row['isbn13'],
                    row['publisher'],
                    '' if row['publication_year'] == '' else str(row['publication_year'])[:4],
                    row['total_editions_count'],
                    row['total_added_by'],
                    row['total_rating_sum'],
                    row['total_rating_count'],
                    row['total_text_reviews_count'],
                    score_5,
                    score_4,
                    score_3,
                    score_2,
                    score_1,
                    score_5_pct,
                    score_4_pct,
                    score_3_pct,
                    score_2_pct,
                    score_1_pct,                    
                    row['average_rating'],
                    row['ratings_count'],
                    row['text_reviews_count'],
                    row['authors_average_rating'],
                    row['translators_ids'],
                    row['translators_names'].replace("'",'').replace('[','').replace(']','').replace('\\u200c',' '),
                    row['translators_average_rating'],
                    row['description'],
                    get_image(en_df.loc[i, 'image_url'],350,en_df.loc[i, 'id']),
                    get_image(row['image_url'],250,row['id'])
                ])
    write_to_csv(books_file_path, books)


In [ ]:
concatenate_orginal_persian_books()
print("Concatenate orignal and persian books")

In [ ]:
def get_score(score):
    score = float(score)
    diff = score - math.floor(score)
    round_score = score
    if diff > 0 and diff <= 0.25:
        round_score = math.floor(score) + 0.25
    elif diff > 0.25 and diff <= 0.5:
        round_score = math.floor(score) + 0.5
    elif diff > 0.5 and diff <= 0.75:
        round_score = math.floor(score) + 0.75
    elif diff >= 0.75:
        round_score = int(math.floor(score) + 1)
    return round_score


def get_template_text(path):
    template_text = ""
    with open(path) as layout:
        template_text = layout.read()    
    return template_text
    

def generate_authors_pages():
    df = pd.read_csv(authors_file_path)
    df.sort_values(by='year',ascending = False,inplace=True)
    
    df_org_books = pd.read_csv(orginal_books_file_path)
    df_books = pd.read_csv(books_file_path)
    df_books.fillna('',inplace = True)
    df.fillna('',inplace = True)
    
    author_layout_template = get_template_text("templates/author_layout.tmp")
    author_about_layout_template = get_template_text("templates/author_about.tmp")
    org_book_layout_template = get_template_text("templates/author_org_book.tmp")
    book_layout_template = get_template_text("templates/persian_book_box.tmp")

    layout_template = get_template_text("templates/layout.tmp")
    box_template = get_template_text("templates/box.tmp")
    authors_list_html = ''
    
    for i in range(len(df) - 1,-1,-1):
        box_html = box_template
        author_page = author_layout_template
        author_about_page = author_about_layout_template

        for col in df.columns:
            value = str(df.loc[i, col])
            if col in ['year','gr_born_at','gr_died_at','']:
                value = conver_to_persian_digits(value)
                
            author_about_page = author_about_page.replace("{"+col+"}", value)
            box_html = box_html.replace("{"+col+"}", value)
        
        geners = ""
        for gener in df.loc[i, "wp_fa_genres"].split("،"):
            geners += f" <span>{gener}</span>"
        box_html = box_html.replace("{genres}", geners)
        box_html = box_html.replace("{author_page}", df.loc[i, 'wp_en_name'].replace(' ','_'))
            
        org_books_html = ""
        books = df_books[df_books.author_id == df.loc[i, "gr_id"]]
        org_books = df_org_books[df_org_books.author_id == df.loc[i, "gr_id"]]
        total_org_books = org_books[['total_added_by','average_rating','total_rating_count','total_rating_sum']].sum()
        
        total_readers = int(total_org_books['total_added_by'])
        total_books = len(org_books.index)
        total_avg_rating = 0
        if total_org_books['total_rating_count'] > 0:
            total_avg_rating = round(total_org_books['total_rating_sum'] / total_org_books['total_rating_count'],2)
        total_rating_count = int(total_org_books['total_rating_count'])
        
        for org_book_index,org_book in org_books.iterrows():
            row = 0
            books_box = ''
            org_book_html = org_book_layout_template
            
            if len(books[books.work_id == org_book.work_id]) == 0:
                continue
           
            language = ''
            description = ''
            for index, book in books[books.work_id == org_book.work_id].iterrows():
                if book.description != '':
                    language = detect(book.description)
                
                if language == 'fa':
                    if len(book.description ) > len(description):
                        description = book.description 
                        
                book_box = book_layout_template
                
                for col in df_books.columns:
                    
                    value = book[col]
                    if col in ['publication_year','original_publication_year']:
                        value = str(value)[:4]
                    if col in ['total_added_by','total_rating_count','total_text_reviews_count','ratings_count','text_reviews_count']:
                        value = format(value,',d')
                    if col in ['score_5','score_4','score_3','score_2','score_1','total_added_by','total_rating_count','total_text_reviews_count','average_rating','ratings_count','text_reviews_count','publication_year','original_publication_year']:
                        value = conver_to_persian_digits(str(value))
                    
                    if row == 0:
                        org_book_html = org_book_html.replace("{"+col+"}", str(value))
                    
                    book_box = book_box.replace("{"+col+"}", str(value))
  
               
                books_box += book_box
                row+=1
            
            org_book_html = org_book_html.replace("{book_description}",description if description != '' else book.description )            
            org_book_html = org_book_html.replace("{star_average_rating}", str(get_score(book.average_rating)))
            org_books_html = org_books_html + org_book_html
            org_books_html = org_books_html.replace("{persian_books}", books_box)
           
        


        author_about_page = author_about_page.replace('{books_total_rating_count}',conver_to_persian_digits(str(format(total_rating_count,',d')))) 
        author_about_page = author_about_page.replace('{books_total_added_by}',conver_to_persian_digits(str(format(total_readers,',d'))))  
        author_about_page = author_about_page.replace('{star_books_average_rating}',str(get_score(total_avg_rating))) 
        
        
        author_page = author_page.replace("{author_about}", author_about_page)
        author_page = author_page.replace("{books}", org_books_html)
        
        
            
        box_html = box_html.replace('{total_average_rating}',conver_to_persian_digits(str(total_avg_rating))) 
        box_html = box_html.replace('{star_total_average_rating}',str(get_score(total_avg_rating))) 
        
        box_html = box_html.replace('{total_rating_count}',conver_to_persian_digits(str(format(total_rating_count,',d')))) 
        box_html = box_html.replace('{total_added_by}',conver_to_persian_digits(str(format(total_readers,',d'))))  
        
        authors_list_html += box_html
        with open(f"output/{df.loc[i, 'wp_en_name'].replace(' ','_')}.html", "w") as file:
            file.write(author_page)
    with open(f"output/index.html", "w") as file:
        file.write(layout_template.replace("{authors}", authors_list_html))                  

        

<h1>Statistics</h1>


In [ ]:
def get_youngest_authors():
    df = pd.read_csv(authors_file_path)
    df['winning_age'] = df.year - df.gr_born_at.str[:4].astype('int32')
    df.sort_values(by=['winning_age', 'year'], inplace=True)
    return df[['year', 'gr_id' , 'wp_en_name', 'wp_fa_name', 'winning_age','image_name']][:5]


def get_oldest_authors():
    df = pd.read_csv(authors_file_path)
    df['winning_age'] = df.year - df.gr_born_at.str[:4].astype('int32')
    df.sort_values(by=['winning_age', 'year'], inplace=True, ascending=False)
    return df[['year', 'wp_en_name', 'wp_fa_name', 'winning_age','image_name']][:5]


def get_based_on_decade():
    df = pd.read_csv(authors_file_path)
    df['decade'] = np.floor((df.year - df.gr_born_at.str[:4].astype('int32')) / 10)
    
def get_most_popular_authors():
    df = pd.read_csv(orginal_books_file_path)
    most_popular_authors = df.groupby(['author_id'])['total_added_by'].sum().sort_values(ascending=False)[:10]

    df1 = pd.read_csv(authors_file_path)

    df_inner = pd.merge(df1, most_popular_authors, left_on='gr_id', right_on='author_id',how = 'inner')
    return df_inner[['year','wp_en_name','wp_fa_name','image_name','total_added_by']].sort_values('total_added_by',ascending=False)

def get_authors_with_most_reviews():
    df = pd.read_csv(orginal_books_file_path)
    most_text_reviews = df.groupby(['author_id'])['total_text_reviews_count'].sum().sort_values(ascending=False)[:10]

    df1 = pd.read_csv(authors_file_path)

    df_inner = pd.merge(df1, most_text_reviews, left_on='gr_id', right_on='author_id',how = 'inner')
    return df_inner[['year','wp_en_name','wp_fa_name','image_name','total_text_reviews_count']].sort_values('total_text_reviews_count',ascending=False)
 
def get_most_popular_books():
    df = pd.read_csv(orginal_books_file_path)
    most_popular_book = df.groupby(['id','title','image_url'], as_index=False)[['author_id','total_added_by']].sum().sort_values(by= ['total_added_by'], ascending=False)[:10]
    df1 = pd.read_csv(authors_file_path)
    df_inner = pd.merge(df1, most_popular_book, left_on='gr_id', right_on='author_id',how = 'inner')
    df_inner.fa_title = ''
    persian_books = pd.read_csv(books_file_path)

    for index,row in df_inner.iterrows():
        df_inner.loc[index,'fa_title'] = persian_books[persian_books.org_id == row.id].sort_values(by='ratings_count',ascending=False)[0:1]['title'].values
    return df_inner[['year','wp_en_name','wp_fa_name','image_name','total_added_by','title','image_url','fa_title']].sort_values('total_added_by',ascending=False)

def get_books_with_most_reviews():
    df = pd.read_csv(orginal_books_file_path)
    most_popular_book = df.groupby(['id','title','image_url'], as_index=False)[['author_id','total_text_reviews_count']].sum().sort_values(by= ['total_text_reviews_count'], ascending=False)[:10]
    df1 = pd.read_csv(authors_file_path)
    df_inner = pd.merge(df1, most_popular_book, left_on='gr_id', right_on='author_id',how = 'inner')
    df_inner.fa_title = ''
    persian_books = pd.read_csv(books_file_path)

    for index,row in df_inner.iterrows():
        df_inner.loc[index,'fa_title'] = persian_books[persian_books.org_id == row.id].sort_values(by='ratings_count',ascending=False)[0:1]['title'].values
    return df_inner[['year','wp_en_name','wp_fa_name','image_name','total_text_reviews_count','title','image_url','fa_title']].sort_values('total_text_reviews_count',ascending=False)

def get_based_on_geners():
    df = pd.read_csv(authors_file_path)
    authors = []
    for index,row in df.iterrows():
        genres = row.wp_fa_genres.split('،')
        for gener in genres:
            authors.append([row.gr_id,gener.strip()])
    df = pd.DataFrame(data=authors, columns=["author_id", "gener"])      
    return df.groupby(['gener'], as_index=False).count().sort_values(by='author_id',ascending=False)

In [ ]:
def get_template_text(path):
    template_text = ""
    with open(path) as layout:
        template_text = layout.read()    
    return template_text

def generate_statistics_page():
    list_box_layout = get_template_text("templates/listsbox.tmp")
    
    df = get_youngest_authors()
    rows = ""
    for index,row in df.iterrows():
        row = f"<tr><td><img src='images/authors/{row.image_name}' alt='{row.wp_fa_name}'/></td><td class='text-right'><a href='{row.wp_en_name.replace(' ','_')}.html'>{row.wp_fa_name}</a></td><td>{conver_to_persian_digits(row.winning_age)} سالگی</td></tr>"
        rows+=row
    youngest_authors = list_box_layout
    youngest_authors = youngest_authors.replace("{title}","جوان‌ترین برندگان نوبل ادبیات")
    youngest_authors = youngest_authors.replace("{rows}",rows)
    youngest_authors = youngest_authors.replace("{col_count}","6")
    
    df = get_oldest_authors()
    rows = ""
    for index,row in df.iterrows():
        row = f"<tr><td><img src='images/authors/{row.image_name}' alt='{row.wp_fa_name}'/></td><td class='text-right'><a href='{row.wp_en_name.replace(' ','_')}.html'>{row.wp_fa_name}</a></td><td>{conver_to_persian_digits(row.winning_age)} سالگی</td></tr>"
        rows+=row
    oldest_authors = list_box_layout
    oldest_authors = oldest_authors.replace("{title}","مسن‌ترین برندگان نوبل ادبیات")
    oldest_authors = oldest_authors.replace("{rows}",rows)
   
    oldest_authors = oldest_authors.replace("{col_count}","6")
    
    df = get_most_popular_authors()
    rows = ""
    for index,row in df.iterrows():
        row = f"<tr><td><img src='images/authors/{row.image_name}' alt='{row.wp_fa_name}'/></td><td class='text-right'><a href='{row.wp_en_name.replace(' ','_')}.html'>{row.wp_fa_name}</a></td><td>{conver_to_persian_digits(format(row.total_added_by,',d'))} بار خوانده شده</td></tr>"
        rows+=row
    most_popular_authors = list_box_layout
    most_popular_authors = most_popular_authors.replace("{title}","برندگانی با بیشترین آثار خوانده شده")
    most_popular_authors = most_popular_authors.replace("{rows}",rows)
    most_popular_authors = most_popular_authors.replace("{col_count}","6")

    df = get_authors_with_most_reviews()
    rows = ""
    for index,row in df.iterrows():
        row = f"<tr><td><img src='images/authors/{row.image_name}' alt='{row.wp_fa_name}'/></td><td class='text-right'><a href='{row.wp_en_name.replace(' ','_')}.html'>{row.wp_fa_name}</a></td><td>{conver_to_persian_digits(format(row.total_text_reviews_count,',d'))} ریویو متنی</td></tr>"
        rows+=row
    authors_with_most_reviews = list_box_layout
    authors_with_most_reviews = authors_with_most_reviews.replace("{title}","برندگانی با بیشترین ریویو متنی در آثار")
    authors_with_most_reviews = authors_with_most_reviews.replace("{rows}",rows)
    authors_with_most_reviews = authors_with_most_reviews.replace("{col_count}","6")

    df = get_most_popular_books()
    rows = ""
    for index,row in df.iterrows():
        row = f"<tr><td><img src='images/authors/{row.image_name}' alt='{row.wp_fa_name}'/></td><td class='text-right'><a href='{row.wp_en_name.replace(' ','_')}.html'>{row.wp_fa_name}</a></td><td><img src='{row.image_url}'/></td><td class='text-right'>{row.fa_title}</td><td>{conver_to_persian_digits(format(row.total_added_by,',d'))} بار خوانده شده</td></tr>"
        rows+=row
    most_popular_books = list_box_layout
    most_popular_books = most_popular_books.replace("{title}","آثار بیشتر خوانده شده")
    most_popular_books = most_popular_books.replace("{rows}",rows)
    most_popular_books = most_popular_books.replace("{col_count}","12")
    
    df = get_books_with_most_reviews()
    rows = ""
    for index,row in df.iterrows():
        row = f"<tr><td><img src='images/authors/{row.image_name}' alt='{row.wp_fa_name}'/></td><td class='text-right'><a href='{row.wp_en_name.replace(' ','_')}.html'>{row.wp_fa_name}</a></td><td><img src='{row.image_url}'/></td><td class='text-right'>{row.fa_title}</td><td>{conver_to_persian_digits(format(row.total_text_reviews_count,',d'))} ریویو متنی</td></tr>"
        rows+=row
    books_with_most_reviews = list_box_layout
    books_with_most_reviews = books_with_most_reviews.replace("{title}","آثاری با بیشترین ریویو متنی")
    books_with_most_reviews = books_with_most_reviews.replace("{rows}",rows)
    books_with_most_reviews = books_with_most_reviews.replace("{col_count}","12")
    
    df = get_based_on_geners()
    rows = ""
    for index,row in df.iterrows():
        row = f"<tr><td class='text-right'><td class='text-right'>{row.gener}</td><td>{conver_to_persian_digits(format(row.author_id,',d'))} نویسنده</td></tr>"
        rows+=row
    geners = list_box_layout
    geners = geners.replace("{title}","آثاری با بیشترین ریویو متنی")
    geners = geners.replace("{rows}",rows)
    geners = geners.replace("{col_count}","6")
    
    lists = youngest_authors + oldest_authors + most_popular_authors + authors_with_most_reviews + most_popular_books + books_with_most_reviews + geners
    statistics_layout = get_template_text("templates/statistics_layout.tmp")
    statistics_layout = statistics_layout.replace('{lists}',lists)
    with open(f"output/statistics.html", "w") as file:
        file.write(statistics_layout)   
    


In [ ]:
generate_authors_pages()
generate_statistics_page()
